In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) #to ignore pandas warning
import numpy as np
import pandas as pd
from NeuralNetwork import *
from utils import *


## Data Processing

### Applying One-Hot Encoding and Shuffling

In [2]:
col_names = ["Class", "a1", "a2", "a3", "a4", "a5", "a6", "ID"]
tr_data = pd.read_csv("../monk+s+problems/monks-1.train", sep =" ",  names = col_names)
test_data = pd.read_csv("../monk+s+problems/monks-1.test", sep =" ",  names = col_names)

tr_data = process_monk_data(tr_data)
test_data = process_monk_data(test_data)

print(tr_data)

     Class        ID  a1_1  a1_2  a1_3  a2_1  a2_2  a2_3  a3_1  a3_2  a4_1  \
0        1   data_42     1     0     0     1     0     0     0     1     0   
1        0  data_157     0     1     0     1     0     0     1     0     0   
2        0  data_143     1     0     0     0     0     1     0     1     0   
3        0   data_93     1     0     0     0     1     0     0     1     0   
4        1  data_197     0     1     0     0     1     0     1     0     1   
..     ...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
119      1  data_212     0     1     0     0     1     0     1     0     0   
120      1  data_206     0     1     0     0     1     0     1     0     0   
121      1  data_414     0     0     1     0     0     1     0     1     1   
122      1  data_153     0     1     0     1     0     0     1     0     0   
123      0  data_144     1     0     0     0     0     1     0     1     0   

     a4_2  a4_3  a5_1  a5_2  a5_3  a5_4  a6_1  a6_2  
0       0

## Creating the Neural Network

In [3]:

'''net = NeuralNetwork()

n_inputs = tr_data.shape[1] - 2

net.add_input_layer(n_inputs)

net.add_hidden_layer(n_inputs, 4)

net.add_output_layer(4, 1)
'''

'net = NeuralNetwork()\n\nn_inputs = tr_data.shape[1] - 2\n\nnet.add_input_layer(n_inputs)\n\nnet.add_hidden_layer(n_inputs, 4)\n\nnet.add_output_layer(4, 1)\n'

## Training

### Training with Mean Square Error

In [4]:
hyperparams = {"epochs": 500,
               "eta": 0.5,
               "clip_value": 1000,
               "hid_act_fun": "tanh",
               "out_act_fun": "sigmoid",
               "cost_fun": "mse",
               "mb" : 16,
               "momentum" : 0.01}


#losses = net.train(tr_data, hyperparams)  # TANH IS FASTER?? SCRITTO SULLE SLIDE
# print(net)
# print(net.input_layer.weights)

#plot_loss(losses, cost_fun = hyperparams["cost_fun"])

### Training with Binary Cross Entropy

In [5]:
'''
#RESET THE NETWORK!!!

losses = net.train(tr_data_X, tr_data_Y, epochs=1000,
                   eta=0.5, clip_value=None, hid_act_fun="tanh", out_act_fun="sigmoid", cost_fun="b_ce")  # TANH IS FASTER?? SCRITTO SULLE SLIDE
# print(net)
# print(net.input_layer.weights)

plot_loss(losses, cost_fun="b_ce")
'''

'\n#RESET THE NETWORK!!!\n\nlosses = net.train(tr_data_X, tr_data_Y, epochs=1000,\n                   eta=0.5, clip_value=None, hid_act_fun="tanh", out_act_fun="sigmoid", cost_fun="b_ce")  # TANH IS FASTER?? SCRITTO SULLE SLIDE\n# print(net)\n# print(net.input_layer.weights)\n\nplot_loss(losses, cost_fun="b_ce")\n'

# Test Monk 1

In [6]:
#best_eta = net.k_fold(3, tr_data, hyperparams, "eta", [0.01, 0.05, 0.1,0.5, 0.8, 1])
#print(best_eta)

In [10]:
grid = {
    "eta" : [0.01, 0.1, 0.2, 0.4, 0.5, 0.7, 0.8 ,0.9, 1],
    "mb" : [4,8,16,32, tr_data.shape[0]],
    "momentum" : [1, 0.99, 0.95,  0.9],
    "n_layers" : [1],
    "n_neurons" : [3,4,5],
    "epochs" : [400],
    "clip_value" : [None],
    "hid_act_fun" : ["tanh"],
    "out_act_fun" : ["sigmoid"],
    "cost_fun" : ["b_ce"],
    "ridge_lambda": [None],
    "lasso_lambda": [None],
    "linear_decay": [None],
    "eta0": [None],
    "max_steps": [None],
    "epochs_update": [None]}
    

search_space = get_search_space(grid)

print(len(search_space))
print(search_space)
grid_results = parallel_grid_search(5, tr_data, search_space, n_inputs = tr_data.shape[1] - 2, n_outputs=1)

print(grid_results)
'''
'''


540
[{'eta': 0.01, 'mb': 4, 'momentum': 1, 'n_layers': 1, 'n_neurons': 3, 'epochs': 400, 'clip_value': None, 'hid_act_fun': 'tanh', 'out_act_fun': 'sigmoid', 'cost_fun': 'b_ce', 'ridge_lambda': None, 'lasso_lambda': None, 'linear_decay': None, 'eta0': None, 'max_steps': None, 'epochs_update': None}, {'eta': 0.01, 'mb': 4, 'momentum': 1, 'n_layers': 1, 'n_neurons': 4, 'epochs': 400, 'clip_value': None, 'hid_act_fun': 'tanh', 'out_act_fun': 'sigmoid', 'cost_fun': 'b_ce', 'ridge_lambda': None, 'lasso_lambda': None, 'linear_decay': None, 'eta0': None, 'max_steps': None, 'epochs_update': None}, {'eta': 0.01, 'mb': 4, 'momentum': 1, 'n_layers': 1, 'n_neurons': 5, 'epochs': 400, 'clip_value': None, 'hid_act_fun': 'tanh', 'out_act_fun': 'sigmoid', 'cost_fun': 'b_ce', 'ridge_lambda': None, 'lasso_lambda': None, 'linear_decay': None, 'eta0': None, 'max_steps': None, 'epochs_update': None}, {'eta': 0.01, 'mb': 4, 'momentum': 0.99, 'n_layers': 1, 'n_neurons': 3, 'epochs': 400, 'clip_value': None, 

KeyboardInterrupt: 

In [3]:
params = {
    "eta" : 0.2,
    "mb" : 1,
    "momentum" : 0.6,
    "n_layers" : 1,
    "n_neurons" : 4,
    "epochs" : 400,
    "clip_value" : None,
    "hid_act_fun" : "tanh",
    "out_act_fun" : "sigmoid",
    "cost_fun" : "mse",
    "ridge_lambda": None,
    "lasso_lambda": None,
    "linear_decay": None,
    "eta0": None,
    "max_steps": None,
    "epochs_update": None}

best_net, var, bias = compare_models(2, tr_data, params, n_inputs = tr_data.shape[1] - 2, n_outputs=1)


HIDDEN LAYER:NODE 0 WEIGHTS = -0.34072874062756836, 0.26568966789377996, -0.29246884455709965, -0.0589575613170768, -0.10634433955045516, 0.4022442604013619, 0.6526189771877111, 0.05263683771053995, -0.3781287790002995, 0.04681248575320893, -0.1456889637071068, -0.12562111001330922, -0.0040817199548128635, 0.0054923726643344205, -0.27369386725017425, -0.41183831435222346, -0.4123896571840499,  BIAS = 0.0
NODE 1 WEIGHTS = -0.33910948171752525, -0.41226567335580366, 0.12631386606555153, -0.09930535425010675, -0.03762773698926154, 0.14524934365079917, -0.2312272867235657, 0.5094152146295187, 0.06548562225687198, -0.021630602865580786, 0.2396219621701862, -0.001954049814015883, -0.5059896608487618, -0.5321887086472075, 0.08375251788033417, 0.33942388814719693, 0.1360558235164647,  BIAS = 0.0
NODE 2 WEIGHTS = -0.029112478084192768, 0.4933950130941091, 0.11111418663221304, -0.3729453499556172, -0.15275815148433278, -0.5250899409512672, 0.20492551008483287, 0.2674627340615244, 0.7930972796023

In [4]:
print(best_net)
print(var)
print(bias)

INPUT LAYER: 
NODE 0 OUTPUT = 0
NODE 1 OUTPUT = 0
NODE 2 OUTPUT = 1
NODE 3 OUTPUT = 0
NODE 4 OUTPUT = 0
NODE 5 OUTPUT = 1
NODE 6 OUTPUT = 1
NODE 7 OUTPUT = 0
NODE 8 OUTPUT = 0
NODE 9 OUTPUT = 0
NODE 10 OUTPUT = 1
NODE 11 OUTPUT = 0
NODE 12 OUTPUT = 0
NODE 13 OUTPUT = 0
NODE 14 OUTPUT = 1
NODE 15 OUTPUT = 0
NODE 16 OUTPUT = 1

LAYER 0 
NODE 0 WEIGHTS = 0.35174151249730456, 0.5236182030700277, -0.10578792576506874, 0.28019906017052326, 0.11855214766627992, 0.40057456973778033, -0.48092306699434717, 0.17285255180629358, 0.16656031282155037, -0.6146874437972969, -0.07048567796787548, -0.5653878150871865, -0.3729161882202241, -0.6143965499675894, -0.4180353960596259, 0.044045151126576004, -0.46739393569007265,  BIAS = 0.0
NODE 1 WEIGHTS = -0.16805427757446165, -0.3536751628583093, -0.4655418174953031, -0.013374474292509387, -0.12431301634733602, 0.035598413867065075, 0.2367807334595817, -0.0810874730696471, -0.39000195780810487, -0.2539302205585295, 0.2292238485943712, -0.2464867683722082, 